In [1]:
import pandas as pd
import numpy as np
import seaborn as sea
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df=pd.read_csv("Churn_Modelling.csv")
df


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
df.drop(columns=["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [4]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder

label=LabelEncoder()
df["Gender"]=label.fit_transform(df["Gender"])

In [6]:
one_hot=OneHotEncoder()

geo_encoded=one_hot.fit_transform(df[["Geography"]])

In [7]:
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=one_hot.get_feature_names_out(["Geography"]))

In [9]:
data=pd.concat([df.drop("Geography",axis=1),encoded_df],axis=1)

In [10]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [24]:
from sklearn.model_selection import train_test_split

X=data.drop("EstimatedSalary",axis=1)
y=data["EstimatedSalary"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [25]:
import pickle

with open("label_encoder_reg.pkl","wb") as file:
    pickle.dump(label,file)

with open("one_hot_encoder_reg.pkl","wb") as file:
    pickle.dump(one_hot,file)

with open("scaler_reg.pkl","wb") as file:
    pickle.dump(scaler,file)

### ANN Solving

In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [41]:
model=Sequential(
    [
        Dense(64,activation="relu",input_shape=(X_train.shape[1],)),
        Dense(32,activation="relu"),
        Dense(1) #Default activation function=linear
    ]
)

e:\DL_Project_1\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.MeanAbsoluteError()

In [44]:
model.compile(optimizer=opt,loss=loss,metrics=["mae"])


In [45]:
from datetime import datetime

log_dir="reglogs/fit/"+datetime.now().strftime("%Y%m%d-%H%M%S")

tensor_callbacks=TensorBoard(log_dir=log_dir,histogram_freq=1)



In [46]:
early_callbacks=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [47]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensor_callbacks,early_callbacks])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 95191.0547 - mae: 95191.0547 - val_loss: 53130.2109 - val_mae: 53130.2109
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 51348.6016 - mae: 51348.6016 - val_loss: 50368.1484 - val_mae: 50368.1484
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 49947.2969 - mae: 49947.2969 - val_loss: 50234.4375 - val_mae: 50234.4375
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 49929.1875 - mae: 49929.1875 - val_loss: 50146.3125 - val_mae: 50146.3125
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 49552.7773 - mae: 49552.7773 - val_loss: 50199.6641 - val_mae: 50199.6641
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 49993.6875 - mae: 49993.6875 - val_loss: 50239.0859 - val_mae: 50239.0859
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 49393.4141 - mae: 49393.4141 - val_loss: 50240.2812 - val_mae: 50240.2812
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/

In [48]:
%load_ext tensorboard

In [52]:
%tensorboard --logdir reglogs/fit/20250601-165119

Reusing TensorBoard on port 6006 (pid 18604), started 0:00:08 ago. (Use '!kill 18604' to kill it.)

In [50]:
model.save("regmodel.h5")